In [54]:
import os
import pandas as pd
import numpy as np
import re
import datetime
import pickle
import json
import time
from nltk.tokenize import sent_tokenize
import nltk
from collections import Counter,defaultdict

from nltk.stem import WordNetLemmatizer,PorterStemmer
  
lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

os.chdir('..')
print(os.getcwd())
from utils import get_fulltext,get_fname,fulltext_exists
os.chdir('./data_processing/')

/Users/yiweiluo/scientific-debates


In [8]:
import spacy
from spacy import displacy
import en_core_web_sm
nlp = en_core_web_sm.load()

In [14]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [2]:
df = pd.read_pickle('../data_scraping/dedup_combined_df.pkl')
df.shape

(44582, 10)

In [5]:
df.loc[0].url

'www.buzzfeednews.com/article/tasneemnashrulla/eat-babies-aoc-town-hall-pro-trump-troll-larouche'

In [6]:
get_fname(df.loc[0].url)

'www.buzzfeednews.com[SEP]article[SEP]tasneemnashrulla[SEP]eat-babies-aoc-town-hall-pro-trump-troll-larouche'

In [7]:
get_fulltext(df.loc[0].url)

['The president and the far right slammed Rep. Alexandria Ocasio-Cortez over the bizarre town hall speaker who suggested eating babies to solve the climate crisis. But the pro-Trump LaRouche PAC, which believes climate change is a hoax, said it planted the troll. President Donald Trump, his son Don Jr., and far-right conservatives attacked Democratic Rep. Alexandria Ocasio-Cortez after a speaker at her town hall meeting Thursday night went on a bizarre rant about how eating babies is the only solution to the climate crisis. But it turns out the stunt was staged by what is now a far-right pro-Trump conspiracy group that compares climate change activism to “genocide.” Trump called Ocasio-Cortez a “wack job” on Twitter after his allies widely shared the video of the unidentified woman saying that Ocasio-Cortez’s Green New Deal was not enough to solve the climate crisis. AOC is a Wack Job! https://t.co/LU3hIeek0c  “We’re not going to be here for much long because of the climate crisis,” th

# Process each article for instances of NEs

In [10]:
doc = nlp('European authorities fined Google a record $5.1 billion on Wednesday for abusing its power in the \
mobile phone market and ordered the company to alter its practices')
print([(X.text, X.label_) for X in doc.ents])

[('European', 'NORP'), ('Google', 'ORG'), ('$5.1 billion', 'MONEY'), ('Wednesday', 'DATE')]


In [74]:
def get_NEs(art):
    doc = nlp(art)
    doc_persons = set([X.text for X in doc.ents if X.label_=='PERSON'])
    person_clusters = dict()
    for person in doc_persons:
        if person not in person_clusters and len(person) > 2:
            #print(person)
            matches = process.extract(person,doc_persons)
            true_matches = [m for m in matches if m[1] > 80]
            #print(true_matches)
            if len(true_matches) > 0:
                longest = max([x[0] for x in true_matches], key=len)
                #print(longest)
                person_clusters.update({m[0]:longest for m in true_matches})
        
    return set(person_clusters.values())

In [44]:
# Storage format: csv
| NE | original article(s) |

{'@AOC All',
 '@RepAOC',
 'Alexandria Ocasio-Cortez',
 'Daniel Burke  ',
 'Donald Trump',
 'Facebook',
 'Feed Jane Fonda',
 'Greta Thunberg',
 'Henry Kissinger',
 'Jonathan Swift’s',
 'LaRouchies',
 'Lyndon H. LaRouche Jr.',
 'Matthew Sweet',
 'Michael Dukakis',
 'Nazi Youth',
 'Obama',
 'Putin',
 'Queen',
 'Tucker Carlson',
 'https://t.co/LU3hIeek0c  '}

In [ ]:
#NEs_and_sources = defaultdict(list)
for n,url in enumerate(df.url.values[13:]):
    ft = get_fulltext(url)[0]
    if len(ft) > 0:
        NEs = get_NEs(ft)
        #print(url,NEs)
        for NE in NEs:
            NEs_and_sources[NE].append(url)
    if n % 1000 == 0:
        print(n,url)

0 www.foxnews.com/entertainment/30s-star-thelma-todd-mysterious-death


1000 www.breitbart.com/news/arctic-wildfires-continue-to-burn-releasing-record-amounts-of-co2/


2000 www.nytimes.com/2018/10/31/climate/climate-change-vote-midterms.html


3000 www.nytimes.com/2020/02/25/science/congo-river-fish.html


4000 www.breitbart.com/environment/2020/01/21/greta-thunberg-tells-world-to-start-listening-to-her-warnings-of-climate-catastrophe/


...

21000 www.nytimes.com/2015/06/25/opinion/the-popes-encyclical-on-the-environment.html
22000 www.nytimes.com/2018/12/15/opinion/ryan-zinke-interior-resigns-trump.html


23000 www.motherjones.com/environment/2014/09/rebecca-solnit-why-im-marching-climate-change-un/


24000 www.csmonitor.com/Environment/Energy/2015/0319/Obama-cracks-down-on-another-emissions-giant-the-US-government


25000 www.csmonitor.com/Environment/Wildlife/2008/1112/supreme-court-lifts-limits-on-navy-sonar-near-whales


26000 www.nytimes.com/2012/11/15/opinion/standing-tougher.html
27000 www.nytimes.com/2017/06/04/us/politics/trump-paris-accord-bush-kyoto.html


In [72]:
get_NEs(ft)

”  
[]
Marilyn Monroe's
[("Marilyn Monroe's", 100)]
Marilyn Monroe's
Morgan
[('Morgan', 100), ('Michelle Morgan', 90)]
Michelle Morgan
Roland West
[('Roland West', 100)]
Roland West
MOORE
[('MOORE', 100)]
MOORE
Lucky Luciano
[('Lucky Luciano', 100)]
Lucky Luciano
Thelma Todd’s
[('Thelma Todd’s', 100), ('Thelma Todd', 95), ('Todd', 90)]
Thelma Todd’s
Pat DiCicco
[('Pat DiCicco', 100)]
Pat DiCicco
Barbara La
[('Barbara La', 100)]
Barbara La
ZaSu Pitts
[('ZaSu Pitts', 100)]
ZaSu Pitts
Jewel Carmen
[('Jewel Carmen', 100)]
Jewel Carmen
A.F. Wagner
[('A.F. Wagner', 100)]
A.F. Wagner
Donald Gallery
[('Donald Gallery', 100)]
Donald Gallery
Kindle
[('Kindle', 100)]
Kindle


{'A.F. Wagner',
 'Barbara La',
 'Donald Gallery',
 'Jewel Carmen',
 'Kindle',
 'Lucky Luciano',
 'MOORE',
 "Marilyn Monroe's",
 'Michelle Morgan',
 'Pat DiCicco',
 'Roland West',
 'Thelma Todd’s',
 'ZaSu Pitts'}

In [64]:
NEs_and_sources

defaultdict(list,
            {'Greta Thunberg': ['www.buzzfeednews.com/article/tasneemnashrulla/eat-babies-aoc-town-hall-pro-trump-troll-larouche'],
             'Matthew Sweet': ['www.buzzfeednews.com/article/tasneemnashrulla/eat-babies-aoc-town-hall-pro-trump-troll-larouche'],
             'https://t.co/LU3hIeek0c  ': ['www.buzzfeednews.com/article/tasneemnashrulla/eat-babies-aoc-town-hall-pro-trump-troll-larouche'],
             'Donald Trump': ['www.buzzfeednews.com/article/tasneemnashrulla/eat-babies-aoc-town-hall-pro-trump-troll-larouche',
              'www.buzzfeednews.com/article/llevin/opinion-im-18-and-joe-biden-couldnt-answer-my-questions-how'],
             'Michael Dukakis': ['www.buzzfeednews.com/article/tasneemnashrulla/eat-babies-aoc-town-hall-pro-trump-troll-larouche'],
             'LaRouchies': ['www.buzzfeednews.com/article/tasneemnashrulla/eat-babies-aoc-town-hall-pro-trump-troll-larouche'],
             'Putin': ['www.buzzfeednews.com/article/tasneemnashrulla/ea

In [ ]:
pickle.dump(NEs_and_sources,open('NE2src_dict.pkl','wb'))